# Portfolio Optimization Suite
Markowitz · Black–Litterman · Quadratic Transaction Costs

This notebook implements three portfolio optimization models:
- **Markowitz mean–variance**
- **Black–Litterman**
- **Quadratic Transaction Costs (QTC)**

Input data should be placed in the `data/` folder (e.g., `prices.csv`).

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from pathlib import Path

DATA_DIR = Path('data')
PRICES_FILE = DATA_DIR / 'prices.csv'

## 1. Load and preprocess data

In [ ]:
# Load prices
prices = pd.read_csv(PRICES_FILE, parse_dates=['Date'], index_col='Date')
returns = prices.pct_change().dropna()

# Annualization factor (daily data)
ann_factor = 252
mu = returns.mean() * ann_factor
Sigma = returns.cov() * ann_factor

mu, Sigma.head()

## 2. Markowitz mean–variance optimization

In [ ]:
def markowitz_min_variance(mu, Sigma):
    n = len(mu)
    w = cp.Variable(n)
    risk = cp.quad_form(w, Sigma)
    prob = cp.Problem(cp.Minimize(risk), [cp.sum(w) == 1, w >= 0])
    prob.solve()
    return w.value

weights_mv = markowitz_min_variance(mu, Sigma)
weights_mv

## 3. Black–Litterman model

In [ ]:
def black_litterman(mu, Sigma, P, Q, Omega, tau=0.05):
    # Posterior mean
    M = np.linalg.inv(np.linalg.inv(tau*Sigma) + P.T @ np.linalg.inv(Omega) @ P)
    mu_bl = M @ (np.linalg.inv(tau*Sigma) @ mu + P.T @ np.linalg.inv(Omega) @ Q)
    return mu_bl

# Example dummy matrices (replace with real views)
n = len(mu)
P = np.eye(n)
Q = mu.values.reshape(-1,1)
Omega = np.eye(n) * 0.01
mu_bl = black_litterman(mu.values, Sigma.values, P, Q, Omega)
mu_bl

## 4. Quadratic Transaction Costs (QTC)

In [ ]:
def optimize_with_qtc(mu, Sigma, w0, cost_diag, lam_r=1.0, lam_mu=1.0, lam_c=1.0):
    n = len(mu)
    x = cp.Variable(n)  # trades
    w = w0 + x
    risk = cp.quad_form(w, Sigma)
    ret = mu @ w
    cost = cp.quad_form(x, np.diag(cost_diag))
    obj = lam_r * risk - lam_mu * ret + lam_c * cost
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve()
    return w.value

w0 = np.repeat(1/len(mu), len(mu))
cost_diag = np.repeat(0.001, len(mu))
weights_qtc = optimize_with_qtc(mu.values, Sigma.values, w0, cost_diag)
weights_qtc